# App

> Translate an image based on a prompt.  

In [ ]:
#| default_exp app

In [ ]:
#| export
import os
import time
import base64
from io import BytesIO
import threading
import atexit
from PIL import Image
from dotenv import load_dotenv
from flask import Flask, jsonify, request, send_file, current_app, Response, render_template, stream_with_context
import huggingface_hub
from mmm_st.video import VideoStreamer, interpolate_images
from mmm_st.diffuse import get_transformer
from mmm_st.config import Config

In [ ]:
#| export
app = Flask(__name__)

load_dotenv()

huggingface_hub.login(
    token=os.environ.get("HF_TOKEN"),
    add_to_git_credential=True,
)

class SharedResources:
    def __init__(self, transformer_type=None):
        self.transformer_type = transformer_type or Config.TRANSFORMER_TYPE
        self.image_transformer = get_transformer(self.transformer_type)()
        self.transform_lock = threading.Lock()
        self.shutdown_event = threading.Event()
        self.video_streamer = VideoStreamer(0, capture_props=Config.CAP_PROPS, event=self.shutdown_event)
        self.current_frame = None
        self.prompt = None

class TransformationThread(threading.Thread):
    def __init__(self, shared_resources):
        super().__init__(daemon=True)
        self.shared_resources = shared_resources

    def run(self):
        print("Transformation thread starting...")
        self.shared_resources.video_streamer.start()
        self.shared_resources.current_frame = self.shared_resources.video_streamer.get_current_frame()  # Set the first frame immediately
        while not self.shared_resources.shutdown_event.is_set():
            self.process_frame()
            if self.shared_resources.shutdown_event.is_set():
                break
    
        print("Transformation thread exiting.")

    def process_frame(self):
        frame = self.shared_resources.video_streamer.get_current_frame()
        if (frame is not None) and self.shared_resources.prompt:
            with self.shared_resources.transform_lock:
                transformed_image = self.shared_resources.image_transformer(frame, self.shared_resources.prompt)
                # Interpolate between current frame and transformed image
                interpolated_image = interpolate_images(self.shared_resources.current_frame, transformed_image, 0.5)
                self.shared_resources.current_frame = interpolated_image
        elif (frame is not None):
            self.shared_resources.current_frame = frame

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/set_prompt', methods=['POST'])
def set_prompt():
    prompt = request.json.get('prompt')
    if not prompt:
        return jsonify({"error": "Prompt is required"}), 400
    with current_app.shared_resources.transform_lock:
        current_app.shared_resources.prompt = prompt
    return jsonify({"message": "Prompt set successfully"}), 200

@app.route('/get_transformed_image', methods=['GET'])
def get_transformed_image():
    with current_app.shared_resources.transform_lock:
        image = current_app.shared_resources.current_frame
    if image is None:
        return jsonify({"error": "No image available"}), 404
    return serve_pil_image(image)

@app.route('/stream')
def stream():
    def generate():
        # Push the application context
        app_context = current_app.app_context()
        app_context.push()
        
        try:
            while True:
                with current_app.shared_resources.transform_lock:
                    image = current_app.shared_resources.current_frame
                    if image is not None:
                        img_byte_arr = BytesIO()
                        Image.fromarray(image).save(img_byte_arr, format='JPEG')
                        img_byte_arr.seek(0)
                        encoded_img = base64.b64encode(img_byte_arr.getvalue()).decode('utf-8')
                        yield f"data: {encoded_img}\n\n"
                    time.sleep(0.1)  # Adjust this delay based on your frame rate needs.
                if current_app.shared_resources.shutdown_event.is_set():
                    break
        finally:
            # Ensure the application context is popped correctly
            app_context.pop()

    return Response(stream_with_context(generate()), mimetype='text/event-stream')


def serve_pil_image(image):
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='JPEG')
    img_byte_arr.seek(0)
    return send_file(
        img_byte_arr, 
        mimetype='image/jpeg', 
        as_attachment=True, 
        attachment_filename='transformed_image.jpg',
    )

def cleanup(shared_resources, transformation_thread):
    print("Application is shutting down. Cleaning up resources.")
    shared_resources.shutdown_event.set()
    shared_resources.video_streamer.stop()
    transformation_thread.join()

def main(transformer_name: str = Config.TRANSFORM_TYPE, host: str = Config.HOST, port: int = Config.PORT):
    shared_resources = SharedResources(transformer_name)
    app.shared_resources = shared_resources
    transformation_thread = TransformationThread(shared_resources)
    transformation_thread.start()
    atexit.register(cleanup, shared_resources, transformation_thread)
    app.run(host=host, port=port)

In [ ]:
#| export
if __name__ == "__main__":
    main()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()